In [1]:
import random
import copy
import pandas as pd

In [2]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
import numpy as np

In [3]:
deck = [11, 11, 11, 11,
        2, 2, 2, 2, 
        3, 3, 3, 3,
        4, 4, 4, 4,
        5, 5, 5, 5,
        6, 6, 6, 6,
        7, 7, 7, 7,
        8, 8, 8, 8,
        9, 9, 9, 9,
        10, 10, 10, 10,
        10, 10, 10, 10,
        10, 10, 10, 10,
        10, 10, 10, 10,]

In [4]:
# game_status = False
# current_deck, dealer_cards, player_cards, stand_status = start_game(deck)
# print(f"dealer: {dealer_cards}")
# print(f"player: {player_cards}")
# while game_status == False and stand_status == False: 
#     print("Action")
#     input_action = input()
#     player_cards, dealer_cards, current_deck, stand_status = action(input_action, player_cards, dealer_cards, current_deck)
#     evalu, game_status = evaluation(dealer_cards, player_cards, stand_status)
#     print(f"dealer: {dealer_cards}")
#     print(f"player: {player_cards}")
#     print(f"{evalu} {game_status}")
#     if game_status == True:
#         print(f"Outcome: {evalu}")
#         print(f"Game Status: {game_status}")


In [76]:
def deal_card(current_deck, persons_cards):
    card_dealt = random.choice(current_deck)
    persons_cards.append(card_dealt)
    current_deck.remove(card_dealt)
    return persons_cards, current_deck

def hit(current_deck, persons_cards):
    persons_cards, current_deck = deal_card(current_deck, persons_cards)
    if (11 in persons_cards) and (sum(persons_cards) > 21):
        persons_cards = [1 if card == 11 else card for card in persons_cards]
    return persons_cards, current_deck

def stand(current_deck, dealer_cards):
    stand_status = True
    while sum(dealer_cards) <= 16:
        dealer_cards, current_deck = hit(current_deck, dealer_cards)
    return dealer_cards, current_deck, stand_status

def evaluation(dealer_cards, player_cards, stand_status):
    if sum(player_cards) > 21:
        game_status = True
        fitness = -100
        return "Player Bust", game_status, fitness
    elif sum(player_cards) == 21:
        game_status = True
        fitness = sum(player_cards)
        return "Black Jack", game_status, fitness
    elif sum(dealer_cards) > 21: 
        game_status = True
        fitness = sum(player_cards)
        return "Dealer Bust", game_status, fitness
    elif (sum(player_cards) > sum(dealer_cards)) and (stand_status==True):
        game_status = True
        fitness = sum(player_cards)
        return "Player Wins", game_status, fitness
    elif stand_status==False:
        game_status = False
        fitness = None
        return None, game_status, fitness
    elif sum(player_cards) == sum(dealer_cards):
        game_status = True
        fitness = sum(player_cards)
        return "Draw", game_status, fitness
    else:
        game_status = True
        fitness = 0
        return "Dealer Wins", game_status, fitness
    
def player_hit(player_cards, current_deck):
    player_cards, current_deck = hit(current_deck, player_cards)
    if sum(player_cards) > 21:
        return player_cards, current_deck
    else:
        return player_cards, current_deck

# def player_hit_evaluation(player_cards, dealer_cards, current_deck, input_action):
#     player_cards, dealer_cards, current_deck = action(input_action, player_cards, dealer_cards, current_deck)
#     evalu = evaluation(dealer_cards, player_cards)
#     return evalu, player_cards, dealer_cards, current_deck
    
def action(input_action, player_cards, dealer_cards, current_deck):
    if input_action == "H":
        stand_status = False
        player_cards, current_deck = player_hit(player_cards, current_deck)
    elif input_action == "S":
        dealer_cards, current_deck, stand_status = stand(current_deck, dealer_cards)
    return player_cards, dealer_cards, current_deck, stand_status
    
def start_game(deck):
    current_deck = copy.deepcopy(deck)
    stand_status = False
    dealer_cards = []
    player_cards = []

    dealer_cards, current_deck = deal_card(current_deck, dealer_cards)
    player_cards, current_deck = deal_card(current_deck, player_cards)
    player_cards, current_deck = deal_card(current_deck, player_cards)
    return current_deck, dealer_cards, player_cards, stand_status

In [75]:
def create_model():
    model = Sequential()
    model.add(Dense(2, input_shape=(2,)))
    model.add(Activation('relu'))
    model.add(Dense(7, input_shape=(2,)))
    model.add(Activation('relu'))
    model.add(Dense(7, input_shape=(2,)))
    model.add(Activation('relu'))
    model.add(Dense(1, input_shape=(2,)))
    model.add(Activation('sigmoid'))

    model.compile(loss='mse', optimizer='adam')
    return model

def predict_action(sum_player_cards, sum_dealer_cards, model_num):
    global current_pool
    neural_input = np.asarray([sum_player_cards, sum_dealer_cards])
    neural_input = np.atleast_2d(neural_input)

    output_prob = current_pool[model_num].predict(neural_input, verbose=0)[0]
    
    if(output_prob[0] >= .5):
        return  "H", output_prob[0]
    return "S", output_prob[0]
    

def model_crossover(parent1, parent2):
    global current_pool
    
    weight1 = current_pool[parent1].get_weights()
    weight2 = current_pool[parent2].get_weights()
    
    new_weight1 = weight1
    new_weight2 = weight2
    
    gene = random.randint(0, len(new_weight1)-1)
    
    new_weight1[gene] = weight2[gene]
    new_weight2[gene] = weight1[gene]
    return np.asarray([new_weight1, new_weight2])

def model_mutate(weights):
    for i in range(len(weights)):
        for j in range(len(weights[i])):
            if random.uniform(0, 1) > .85:
                change = random.uniform(-.05, .05)
                weights[i][j] += change
    return weights

def evalution_heatmap(best_parent_num):
    global current_pool
    outcome_list = []
    for sum_player_opener in range(4, 21):
        for sum_dealer_opener in range(2, 12):
            input_action, output_prob = predict_action(sum_player_opener, sum_dealer_opener, best_parent_num)
            #outcome_list.append(f"{input_action} {round(output_prob, 2)}")
            outcome_list.append(f"{input_action}")
    outcome_list.reverse()
    outcome_array = np.array(outcome_list).reshape(17,10)
    cols = ["A", 10, 9, 8, 7, 6, 5, 4, 3, 2]
    rows = [20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4]
    df = pd.DataFrame(outcome_array, columns=cols)
    df["player_sum"] = rows
    df = df.set_index("player_sum")
    return df

def game_end():
    global current_pool
    global fitness
    global generation
    new_weights = []

    global highest_fitness
    global best_weights
    updated = False
                    

    for select in range(total_models):

        if fitness[select] >= highest_fitness:
            updated = True
            highest_fitness = fitness[select]
            best_weights = current_pool[select].get_weights()

    parent1 = random.randint(0,total_models-1)
    parent2 = random.randint(0,total_models-1)

    for i in range(total_models):
        if fitness[i] >= fitness[parent1]:
            parent1 = i
    for j in range(total_models):
        if j != parent1:
            if fitness[j] >= fitness[parent2]:
                parent2 = j
    
    if generation == 1 or generation % 2 == 0:
        heatmap = evalution_heatmap(parent1)
    else:
        heatmap = None
        
    for select in range(total_models // 2):
        cross_over_weights = model_crossover(parent1,parent2)
        if updated == False:
            cross_over_weights[1] = best_weights
        mutated1 = model_mutate(cross_over_weights[0])
        mutated2 = model_mutate(cross_over_weights[0])
        new_weights.append(mutated1)
        new_weights.append(mutated2)

    for select in range(len(new_weights)):
        fitness[select] = 0
        current_pool[select].set_weights(new_weights[select])

    generation += 1
    return heatmap

In [65]:
def run_game():
    global total_models
    global current_pool
    global fitness
    global deck
    global generations
    updated = False
    games_agents_plays = 1
    
    for i in range(total_models):
        model = create_model()
        current_pool.append(model)
        fitness.append(0)
        
    for i in range(1, generation_runs+1):
        print(f"=== GEN {i} ===")
        for model_num in range(total_models):
            for game in range(games_agents_plays):
                game_status = False
                stand_status = False
                current_deck, dealer_cards, player_cards, stand_status = start_game(deck)

                while game_status == False and stand_status == False: 
                    input_action, output_prob = predict_action(sum(player_cards), sum(dealer_cards), model_num)
                    player_cards, dealer_cards, current_deck, stand_status = action(input_action, player_cards, dealer_cards, current_deck)
                    evalu, game_status, player_win = evaluation(dealer_cards, player_cards, stand_status)
                    print(f"{output_prob}")
                print(f"Model: {model_num}. Game: {evalu}. \n Player Cards: {player_cards}. Dealer Cards: {dealer_cards}")

                if player_win > 0:
                    fitness[model_num] += (player_win)
        df = game_end()
        print(df)
       
    pass

In [78]:
load_saved_pool = False
save_current_pool = True
current_pool = []
fitness = []
total_models = 4
generation = 1
generation_runs = 100

highest_fitness = -1
best_weights = []


run_game()

=== GEN 1 ===
0.2599499821662903
Model: 0. Game: Player Wins. 
 Player Cards: [10, 10]. Dealer Cards: [6, 2, 9]
0.5
0.5
0.5
Model: 1. Game: Player Bust. 
 Player Cards: [9, 3, 2, 1, 9]. Dealer Cards: [10]
0.15609721839427948
Model: 2. Game: Dealer Bust. 
 Player Cards: [10, 10]. Dealer Cards: [9, 7, 6]
0.5
Model: 3. Game: Player Bust. 
 Player Cards: [10, 6, 10]. Dealer Cards: [2]


C:\Users\Erik\AppData\Local\Temp\ipykernel_39560\2393281491.py:40: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.asarray([new_weight1, new_weight2])


            A 10  9  8  7  6  5  4  3  2
player_sum                              
20          S  S  S  S  S  S  S  S  S  S
19          S  S  S  S  S  S  S  S  S  S
18          S  S  S  S  S  S  S  S  S  S
17          S  S  S  S  S  S  S  S  S  S
16          S  S  S  S  S  S  S  S  S  S
15          S  S  S  S  S  S  S  S  S  S
14          S  S  S  S  S  S  S  S  S  S
13          S  S  S  S  S  S  S  S  S  S
12          S  S  S  S  S  S  S  S  S  S
11          S  S  S  S  S  S  S  S  S  S
10          S  S  S  S  S  S  S  S  S  S
9           S  S  S  S  S  S  S  S  S  S
8           S  S  S  S  S  S  S  S  S  S
7           S  S  S  S  S  S  S  S  S  S
6           S  S  S  S  S  S  S  S  S  S
5           S  S  S  S  S  S  S  S  S  S
4           S  S  S  S  S  S  S  S  S  S
=== GEN 2 ===
0.4803523123264313
Model: 0. Game: Black Jack. 
 Player Cards: [11, 10]. Dealer Cards: [2, 10, 2, 9]
0.4905533492565155
Model: 1. Game: Dealer Wins. 
 Player Cards: [8, 3]. Dealer Cards: [3, 5, 5, 2, 4]
0.41

C:\Users\Erik\AppData\Local\Temp\ipykernel_39560\2393281491.py:40: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.asarray([new_weight1, new_weight2])
C:\Users\Erik\AppData\Local\Temp\ipykernel_39560\2393281491.py:40: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.asarray([new_weight1, new_weight2])


            A 10  9  8  7  6  5  4  3  2
player_sum                              
20          S  S  S  S  S  S  S  S  S  S
19          S  S  S  S  S  S  S  S  S  S
18          S  S  S  S  S  S  S  S  S  S
17          S  S  S  S  S  S  S  S  S  S
16          S  S  S  S  S  S  S  S  S  S
15          S  S  S  S  S  S  S  S  S  S
14          S  S  S  S  S  S  S  S  S  S
13          S  S  S  S  S  S  S  S  S  S
12          S  S  S  S  S  S  S  S  S  S
11          S  S  S  S  S  S  S  S  S  S
10          S  S  S  S  S  S  S  S  S  S
9           S  S  S  S  S  S  S  S  S  S
8           S  S  S  S  S  S  S  S  S  S
7           S  S  S  S  S  S  S  S  S  S
6           S  S  S  S  S  S  S  S  S  S
5           S  S  S  S  S  S  S  S  S  S
4           S  S  S  S  S  S  S  S  S  S
=== GEN 3 ===
0.48665091395378113
Model: 0. Game: Dealer Bust. 
 Player Cards: [10, 8]. Dealer Cards: [4, 10, 10]
0.48504987359046936
Model: 1. Game: Dealer Bust. 
 Player Cards: [9, 10]. Dealer Cards: [2, 10, 2, 10]
0.49

...

C:\Users\Erik\AppData\Local\Temp\ipykernel_39560\2393281491.py:40: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.asarray([new_weight1, new_weight2])
C:\Users\Erik\AppData\Local\Temp\ipykernel_39560\2393281491.py:40: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.asarray([new_weight1, new_weight2])


            A 10  9  8  7  6  5  4  3  2
player_sum                              
20          S  S  S  S  S  S  S  S  S  S
19          S  S  S  S  S  S  S  S  S  S
18          S  S  S  S  S  S  S  S  S  S
17          S  S  S  S  S  S  S  S  S  S
16          S  S  S  S  S  S  S  S  S  S
15          S  S  S  S  S  S  S  S  S  S
14          S  S  S  S  S  S  S  S  S  S
13          S  S  S  S  S  S  S  S  S  S
12          S  S  S  S  S  S  S  S  S  S
11          S  S  S  S  S  S  S  S  S  S
10          S  S  S  S  S  S  S  S  S  S
9           S  S  S  S  S  S  S  S  S  S
8           S  S  S  S  S  S  S  S  S  S
7           S  S  S  S  S  S  S  S  S  S
6           H  S  S  S  S  S  S  S  S  S
5           H  H  H  S  S  S  S  S  S  S
4           H  H  H  H  H  S  S  S  S  S
=== GEN 97 ===
0.28046783804893494
Model: 0. Game: Dealer Wins. 
 Player Cards: [4, 11]. Dealer Cards: [3, 6, 10]
0.24851810932159424
Model: 1. Game: Black Jack. 
 Player Cards: [11, 10]. Dealer Cards: [10, 9]
0.356033712

C:\Users\Erik\AppData\Local\Temp\ipykernel_39560\2393281491.py:40: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.asarray([new_weight1, new_weight2])
C:\Users\Erik\AppData\Local\Temp\ipykernel_39560\2393281491.py:40: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.asarray([new_weight1, new_weight2])


            A 10  9  8  7  6  5  4  3  2
player_sum                              
20          S  S  S  S  S  S  S  S  S  S
19          S  S  S  S  S  S  S  S  S  S
18          S  S  S  S  S  S  S  S  S  S
17          S  S  S  S  S  S  S  S  S  S
16          S  S  S  S  S  S  S  S  S  S
15          S  S  S  S  S  S  S  S  S  S
14          S  S  S  S  S  S  S  S  S  S
13          S  S  S  S  S  S  S  S  S  S
12          S  S  S  S  S  S  S  S  S  S
11          S  S  S  S  S  S  S  S  S  S
10          S  S  S  S  S  S  S  S  S  S
9           S  S  S  S  S  S  S  S  S  S
8           S  S  S  S  S  S  S  S  S  S
7           S  S  S  S  S  S  S  S  S  S
6           H  H  S  S  S  S  S  S  S  S
5           H  H  H  S  S  S  S  S  S  S
4           H  H  H  H  H  S  S  S  S  S
=== GEN 99 ===
0.3659592270851135
Model: 0. Game: Dealer Wins. 
 Player Cards: [10, 3]. Dealer Cards: [9, 6, 5]
0.4523557126522064
Model: 1. Game: Dealer Bust. 
 Player Cards: [4, 2]. Dealer Cards: [5, 9, 9]
0.19722349941

C:\Users\Erik\AppData\Local\Temp\ipykernel_39560\2393281491.py:40: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.asarray([new_weight1, new_weight2])
